In [1]:
from Crypto.PublicKey import RSA
import socket
import MCipher

In [ ]:

prikey = RSA.generate(1024)
pubkey = prikey.publickey()

priFile = open("./clientSignPriKey.pem" , "wb+")
priFile.write(prikey.exportKey("PEM"))
priFile.close()

pubFile = open("./clientSignPubKey.pem" , "wb+")
pubFile.write(pubkey.exportKey("PEM"))
pubFile.close()

In [ ]:

def client_program() :
    host = '127.0.0.1'
    port = 5461
    
    keyReceive = False
    client_socket = socket.socket()
    client_socket.connect((host, port))

    if keyReceive == False :
        client_prikey = MCipher.readPEM("clientSignPriKey.pem")
        server_pubkey = MCipher.readPEM("serverSignPubKey.pem")
        
        key_recv = client_socket.recv(1024)
        key = MCipher.RSADecrypt(client_prikey, key_recv)
        print("key : " , key)
        
        client_socket.send('key exchange Success'.encode())
        iv = client_socket.recv(1024).decode()
        print('iv  :' , iv)
        
        client_socket.send('iv exchange Success'.encode())
        
        keyReceive = True
        
    AES = MCipher.setAES(key, iv)
    
    if keyReceive :
        message = input(" -> ")
        
        while message.lower().strip() != 'bye' :
            
            block = MCipher.makeHashBlock(message,client_prikey)
            block_en = MCipher.AES_Encrypt(AES,block)
            client_socket.send(block_en)
            
            r_data = client_socket.recv(1024)
            d_data = MCipher.AES_Decrypt(AES,r_data)
            data, sign_data = MCipher.separateHashBlcok(d_data)
            if MCipher.verify(server_pubkey, data , sign_data ) : 
                print("Server가 보낸 메시지가 맞습니다. :" ,data)
                message = input(" -> ")
            else : 
                print("Server가 보내지 않았습니다.")
                break
            
            
    
    client_socket.close()
        
    
if __name__ == '__main__' :
    client_program()

key :  thisisbadkeyokeythisisbadkeyokey
iv  : ivisintialvector
 -> aaa
Server가 보낸 메시지가 맞습니다. : bbbb
 -> cccc
